### First, let's parse the MCAT/GPA data (from shemassian's website)

Source: https://www.shemmassianconsulting.com/blog/average-gpa-and-mcat-score-for-every-medical-school

In [66]:
import pandas as pd
import os

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"

# Point to the NEW .html file you just saved
html_path = os.path.join(ROOT_DIR, "data/raw_inputs/Average GPA and MCAT Score for Every Medical School (2025) — Shemmassian Academic Consulting.html") 

# This code should work perfectly now
dfs = pd.read_html(html_path, attrs={'id': 'myTable'})
df_mcat_gpa = dfs[0]
df_mcat_gpa.drop(columns=['Minimum MCAT'], inplace=True)
df_mcat_gpa.sort_values(by='Average MCAT', inplace=True, ascending=False)
df_mcat_gpa # Drop any rows where Average MCAT is not a number
df_mcat_gpa = df_mcat_gpa[pd.to_numeric(df_mcat_gpa['Average MCAT'], errors='coerce').notnull()]
# Same for 'Average GPA'.
# Then we want two-level sorting: first by Average MCAT (descending), then by Average GPA (descending)
df_mcat_gpa = df_mcat_gpa[pd.to_numeric(df_mcat_gpa['Average GPA'], errors='coerce').notnull()]
df_mcat_gpa['Average MCAT'] = pd.to_numeric(df_mcat_gpa['Average MCAT'])
df_mcat_gpa['Average GPA'] = pd.to_numeric(df_mcat_gpa['Average GPA'])
df_mcat_gpa.sort_values(by=['Average MCAT', 'Average GPA'], inplace=True, ascending=[False, False])
df_mcat_gpa.reset_index(drop=True, inplace=True)
# Convert 'Medical School' to lower case for easier merging later
df_mcat_gpa['Medical School'] = df_mcat_gpa['Medical School'].str.lower()
df_mcat_gpa['Medical School'] = df_mcat_gpa['Medical School'].str.replace('*', '', regex=False).str.strip()

### Next, lets parse Debt Information from MSAR

Source: https://students-residents.aamc.org/medical-school-admission-requirements/medical-school-admission-requirements-reports-applicants-and-advisors

In [67]:
import camelot
import pandas as pd
import os

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"

file_path = os.path.join(ROOT_DIR, 'data/raw_inputs/MSAR014 - MSAR Debt  Information.pdf')

# 1. Change pages to the full range
tables = camelot.read_pdf(file_path, pages='2-7', flavor='lattice')

processed_dfs = [] # 2. Create a list to hold all our DFs

if tables.n > 0:
    print(f"Found {tables.n} tables across pages 2-7.")
    
    # 3. Loop through every table object
    for table in tables:
        my_df = table.df
        
        # Apply your exact header logic to each table
        try:
            my_df.columns = my_df.iloc[0]
            my_df = my_df[1:]
            processed_dfs.append(my_df)
        except Exception as e:
            print(f"Warning: Could not process a table, possibly bad format? Error: {e}")
            print(f"Problematic table data:\n{table.df.head()}\n")
    
    # 4. Stack all the individual DataFrames into one
    if processed_dfs:
        df_debt_info = pd.concat(processed_dfs, ignore_index=True)
    else:
        df_debt_info = pd.DataFrame() # Empty
        
else:
    print("No tables found in that page range.")
    df_debt_info = pd.DataFrame()

df_debt_info['Average Graduate\nIndebtedness'] = pd.to_numeric(df_debt_info['Average Graduate\nIndebtedness'].str.replace('[\$,]', '', regex=True), errors='coerce')
df_debt_info = df_debt_info[df_debt_info['Average Graduate\nIndebtedness'].notnull()]
df_debt_info.sort_values(by='Average Graduate\nIndebtedness', ascending=True, inplace=True)
df_debt_info.reset_index(drop=True, inplace=True)
df_debt_info.head(30)

# Let's replace any '*' characters in the 'Medical School' column with an empty string
df_debt_info['Medical School'] = df_debt_info['Medical School'].str.replace('*', '', regex=False).str.strip()
# Now turn the 'Medical School' to lower case for easier matching later
df_debt_info['Medical School'] = df_debt_info['Medical School'].str.lower()


<>:40: SyntaxWarning: invalid escape sequence '\$'
<>:40: SyntaxWarning: invalid escape sequence '\$'
/var/folders/76/y_3kwb_54mx0bh961wd_ycb00000gn/T/ipykernel_40148/2777296739.py:40: SyntaxWarning: invalid escape sequence '\$'
  df_debt_info['Average Graduate\nIndebtedness'] = pd.to_numeric(df_debt_info['Average Graduate\nIndebtedness'].str.replace('[\$,]', '', regex=True), errors='coerce')


Found 6 tables across pages 2-7.


### Next, let's parse NIH funding data from Blue Ridge Institute for Medical Research

Source: https://brimr.org/

In [143]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/SchoolOfMedicine_2024_C.xlsx")
df_nih_funding = pd.read_excel(file_path, sheet_name='2024', header=1)
# Currently the 'Name' column is in caps lock, let's make it lower case
df_nih_funding['Name'] = df_nih_funding['Name'].str.lower()
df_nih_funding.head(25)

# However, we're gonna have to add info for some schools manually. Harvard comes to mind (mass general). 

manual_entries_2024 = [
    {'Name': 'massachusetts general hospital', 'School of Medicine Award': 655235087},
    {'Name': 'brigham and womens hospital', 'School of Medicine Award': 388162121},
    {'Name': 'boston childrens hospital', 'School of Medicine Award': 229894668},
    {'Name': 'beth israel deaconess medical center', 'School of Medicine Award': 119860535}
]
df_manual_2024 = pd.DataFrame(manual_entries_2024)
df_nih_funding = pd.concat([df_nih_funding, df_manual_2024], ignore_index=True)
df_nih_funding.reset_index(drop=True, inplace=True)
df_nih_funding


,Rank,Name,School of Medicine Award,Direct Costs,Indirect Costs,Direct Costs as % of Total,Indirect Costs as % of Total,Indirect Costs as % of Direct Costs
0,1.0,university of california san francisco,726020299.0,539756014.0,188866089.0,74.344480,25.655520,34.991012
1,2.0,washington university st louis,683318054.0,512595847.0,176303541.0,75.015704,24.984296,34.394259
2,3.0,yale university,598157843.0,433446452.0,186142364.0,72.463557,27.536443,42.944720
3,4.0,university of pennsylvania,587526359.0,417155739.0,176115805.0,71.002047,28.997953,42.218239
4,5.0,vanderbilt university,586273695.0,442042601.0,155666431.0,75.398676,24.601324,35.215255
...,...,...,...,...,...,...,...,...
153,NaN,median,55623954.5,39936759.0,15649722.5,71.797770,28.202230,71.797770
154,NaN,massachusetts general hospital,655235087.0,NaN,NaN,NaN,NaN,NaN
155,NaN,brigham and womens hospital,388162121.0,NaN,NaN,NaN,NaN,NaN
156,NaN,boston childrens hospital,229894668.0,NaN,NaN,NaN,NaN,NaN


### Next, lets parse the Faculty Data from AAMC
Source: https://www.aamc.org/data-reports/faculty-institutions/report/faculty-roster-us-medical-school-faculty

In [150]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/U.S. Medical School Faculty, 2024.xlsx")
df_faculty_info = pd.read_excel(file_path, sheet_name='USMSF Table 2', header=3)
df_faculty_info = df_faculty_info.iloc[:-3]
df_faculty_info.rename(columns={'Total': 'Total Faculty'}, inplace=True)

### Next, let's parse the USNWR Hospital Rankings Data

Got using the usnwr_scraper.ipynb

In [58]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_hospital_rankings.csv")
df_hospital_rankings_raw = pd.read_csv(file_path)
file_path_pivoted_path = os.path.join(ROOT_DIR, "data/hospital_rankings_pivoted.csv")
df_hospital_rankings_pivoted = pd.read_csv(file_path_pivoted_path)
df_hospital_rankings_pivoted # Let's set the 'Institution' to be all lower case
df_hospital_rankings_pivoted['Institution'] = df_hospital_rankings_pivoted['Institution'].str.lower()
df_hospital_rankings_pivoted

,Institution,Cancer,"Cardiology, Heart & Vascular Surgery",Diabetes & Endocrinology,"Ear, Nose & Throat",Gastroenterology & GI Surgery,Geriatrics,Neurology & Neurosurgery,Obstetrics & Gynecology,Ophthalmology,Orthopedics,Psychiatry,Pulmonology & Lung Surgery,Rehabilitation,Rheumatology,Urology,#n_ranked_specialties,#n_top10_specialties,#n_top1_specialties
0,johns hopkins hospital,8.000000,10.0,6.0,6.0,6.000000,10.000000,7.000000,2.0,3.0,22.0,4.0,11.0,24.0,1.0,8.0,15,12,1
1,new york presbyterian university hospital of c...,15.000000,5.0,2.0,10.0,11.000000,6.000000,3.000000,4.0,NaN,4.0,3.0,10.0,10.0,NaN,4.0,13,11,0
2,nyu langone medical center,16.000000,1.0,4.0,20.0,4.000000,1.000000,1.000000,12.0,NaN,2.0,NaN,1.0,5.0,9.0,2.0,13,10,4
3,mayo clinic,16.333333,29.0,18.0,21.0,11.333333,16.666667,14.333333,3.0,NaN,24.0,NaN,18.0,6.0,4.0,22.0,13,3,0
4,cleveland clinic,14.000000,3.0,28.0,16.0,3.000000,5.000000,13.000000,8.0,10.0,17.0,NaN,6.0,NaN,2.0,9.0,13,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,ohio state university james cancer hospital,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
154,ochsner medical center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
155,baylor institute for rehabilitation at frisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,1,0,0
156,inova fairfax hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0


### Next, Let's Parse Diversity Data

**Source: https://www.aamc.org/data-reports/data/2024-facts**, Table B-5.1

In [181]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/2024_FACTS_Table_B-5.1 - Updated.xlsx")
df_diversity_info = pd.read_excel(file_path, sheet_name='FACTS Table B-5.1', header=4)
df_diversity_info = df_diversity_info[df_diversity_info['Total \nEnrollment'].notnull()]
df_diversity_info.rename(columns={'Total Enrollment': 'State', 'Unnamed: 1': 'AAMC_Institution'}, inplace=True)
df_diversity_info.rename(columns={'Total \nEnrollment': 'Total Enrollment'}, inplace=True)
# We want to make a column called URM%, which is the summed of 'American Indian \nor Alaska Native', 'Black or \nAfrican American', 'Hispanic, Latino, or \nof Spanish Origin', 'Native Hawaiian or \nOther Pacific Islander', and then divided by 'Total Enrollment'
df_diversity_info['American Indian \nor Alaska Native'] = pd.to_numeric(df_diversity_info['American Indian \nor Alaska Native'], errors='coerce').fillna(0)
df_diversity_info['Black or \nAfrican American'] = pd.to_numeric(df_diversity_info['Black or \nAfrican American'], errors='coerce').fillna(0)
df_diversity_info['Hispanic, Latino, or \nof Spanish Origin '] = pd.to_numeric(df_diversity_info['Hispanic, Latino, or \nof Spanish Origin '], errors='coerce').fillna(0)
df_diversity_info['Native Hawaiian or \nOther Pacific Islander'] = pd.to_numeric(df_diversity_info['Native Hawaiian or \nOther Pacific Islander'], errors='coerce').fillna(0)
df_diversity_info['Total Enrollment'] = pd.to_numeric(df_diversity_info['Total Enrollment'], errors='coerce').fillna(1) # Avoid division by zero
df_diversity_info['URM%'] = (
    df_diversity_info['American Indian \nor Alaska Native'] +
    df_diversity_info['Black or \nAfrican American'] +
    df_diversity_info['Hispanic, Latino, or \nof Spanish Origin '] +
    df_diversity_info['Native Hawaiian or \nOther Pacific Islander']
) / df_diversity_info['Total Enrollment'] * 100

df_diversity_info.dropna(subset=['AAMC_Institution'], inplace=True)
df_diversity_info

,State,AAMC_Institution,American Indian \nor Alaska Native,Asian,Black or \nAfrican American,"Hispanic, Latino, or \nof Spanish Origin",Native Hawaiian or \nOther Pacific Islander,White,Other,Multiple \nRace/Ethnicity,Unknown \nRace/Ethnicity,Non-U.S. Citizen and Non-Permanent Resident,Total Enrollment,URM%
1,AL,Alabama-Heersink,1.0,146.0,75.0,29.0,0.0,472.0,18.0,72.0,4.0,0.0,817.0,12.851897
2,NaN,South Alabama-Whiddon,1.0,39.0,36.0,8.0,0.0,187.0,3.0,32.0,0.0,0.0,306.0,14.705882
3,AR,Arkansas,0.0,95.0,30.0,19.0,0.0,511.0,13.0,59.0,1.0,0.0,728.0,6.730769
4,AZ,Arizona,7.0,114.0,21.0,66.0,1.0,190.0,16.0,108.0,1.0,0.0,524.0,18.129771
5,NaN,Arizona Phoenix,2.0,143.0,23.0,36.0,0.0,208.0,16.0,73.0,2.0,0.0,503.0,12.127237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,NaN,Washington State-Floyd,1.0,95.0,10.0,14.0,0.0,166.0,4.0,53.0,1.0,0.0,344.0,7.267442
155,WI,MC Wisconsin,0.0,218.0,76.0,58.0,1.0,616.0,17.0,109.0,4.0,25.0,1124.0,12.010676
156,NaN,Wisconsin,7.0,145.0,60.0,55.0,0.0,375.0,10.0,93.0,2.0,8.0,755.0,16.158940
157,WV,Marshall-Edwards,1.0,43.0,17.0,2.0,0.0,230.0,6.0,23.0,3.0,0.0,325.0,6.153846


### Next, let's parse class size (2025)
Source: https://www.aamc.org/data-reports/students-residents/data/facts-applicants-and-matriculants, Table A-1

In [230]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/2025_FACTS_Table_A-1.xlsx")
df_class_size_info = pd.read_excel(file_path, sheet_name='FACTS Table A-1', header=4)
df_class_size_info = df_class_size_info[df_class_size_info['Matriculants'].notnull()]
df_class_size_info.drop(columns=['Applications'], inplace=True)
df_class_size_info.rename(columns={'Matriculants': 'Class Size', 'Applications by School': 'State', 'Unnamed: 1': 'name', 'Applications1':'Applications'}, inplace=True)
df_class_size_info = df_class_size_info[['name', 'Applications', 'Class Size']]
df_class_size_info.dropna(subset=['name'], inplace=True)
df_class_size_info['Matriculation Rate'] = df_class_size_info['Class Size'] / df_class_size_info['Applications'] * 100
df_class_size_info.sort_values(by='Matriculation Rate', ascending=True, inplace=True)
df_class_size_info['name'].unique().tolist()

['NYU Long Island-Grossman',
 'Kaiser Permanente-Tyson',
 'George Washington',
 'UT Tyler',
 'Virginia Tech Carilion',
 'UT Austin-Dell',
 'Drew',
 'Stanford',
 'Dartmouth-Geisel',
 'TCU-Burnett',
 'Albany',
 'Quinnipiac-Netter',
 'Penn State',
 'Einstein',
 'Emory',
 'Rush',
 'Nova Southeastern-Patel',
 'Houston-Fertitta',
 'UT Rio Grande Valley',
 'Vermont-Larner',
 'Chicago-Pritzker',
 'NYU-Grossman',
 'BU-Chobanian Avedisian',
 'Central Michigan',
 'Cornell-Weill',
 'Vanderbilt',
 'Southern Cal-Keck',
 'Pittsburgh',
 'Loyola-Stritch',
 'Tufts',
 'Duke',
 'Mount Sinai-Icahn',
 'UC San Diego',
 'Tulane',
 'UC Riverside',
 'Georgetown',
 'UCLA-Geffen',
 'Yale',
 'Howard',
 'Meharry',
 'Western Michigan-Stryker',
 'Johns Hopkins',
 'Wake Forest',
 'UC Irvine',
 'Brown-Alpert',
 'Temple-Katz',
 'Morehouse',
 'Arizona',
 'Northwestern-Feinberg',
 'Michigan',
 'Oakland Beaumont',
 'UC Davis',
 'Chicago Med Franklin',
 'UC San Francisco',
 'Colorado',
 'Rochester',
 'Geisinger Commonwealth

### Figure out how to join all the data together

In [235]:
import pandas as pd
from functools import reduce
import json
import numpy as np

# Let's list all the dfs we have so far, along with the key column to join on

dataframes = {
    'MCAT_GPA': [df_mcat_gpa, 'Medical School'],
    'Debt_Info': [df_debt_info, 'Medical School'],
    'NIH_Funding': [df_nih_funding, 'Name'],
    'Faculty_Info': [df_faculty_info, 'Medical School'],
    'Hospital_Rankings': [df_hospital_rankings_pivoted, 'Institution'],
    'Diversity': [df_diversity_info, 'AAMC_Institution'],
    'Class_Size': [df_class_size_info, 'name']
}


# Your data (assuming they are loaded)
df_mcat_gpa = dataframes['MCAT_GPA'][0]
df_debt_info = dataframes['Debt_Info'][0]
df_nih_funding = dataframes['NIH_Funding'][0]
df_faculty_info = dataframes['Faculty_Info'][0]
df_hospital_rankings = dataframes['Hospital_Rankings'][0]
df_diversity_info = dataframes['Diversity'][0]
df_class_size_info = dataframes['Class_Size'][0]

# 1. This is your "ground truth" list
canonical_schools = df_faculty_info['Medical School'].unique()

# 2. This is the "messy" list of *all* names from the other sources
messy_names = pd.concat([
    df_debt_info['Medical School'],
    df_nih_funding['Name'],
    df_mcat_gpa['Medical School'],
    df_hospital_rankings['Institution'],
    df_diversity_info['AAMC_Institution'],
    df_class_size_info['name']
]).unique()

messy_names.tolist()

with open (os.path.join(ROOT_DIR, 'data/names_mapping.json'), 'r') as f:
    name_map = json.load(f)

for school in canonical_schools:
    name_map[school] = school

df_mcat_gpa['canonical_name'] = df_mcat_gpa['Medical School'].map(name_map)
df_mcat_gpa.dropna(subset=['canonical_name'], inplace=True)
df_debt_info['canonical_name'] = df_debt_info['Medical School'].map(name_map)
df_debt_info.dropna(subset=['canonical_name'], inplace=True)
df_nih_funding['canonical_name'] = df_nih_funding['Name'].map(name_map)
df_nih_funding.dropna(subset=['canonical_name'], inplace=True)
df_faculty_info['canonical_name'] = df_faculty_info['Medical School'].map(name_map)
df_faculty_info.dropna(subset=['canonical_name'], inplace=True)
df_hospital_rankings['canonical_name'] = df_hospital_rankings['Institution'].map(name_map)
df_hospital_rankings.dropna(subset=['canonical_name'], inplace=True)
df_diversity_info['canonical_name'] = df_diversity_info['AAMC_Institution'].map(name_map)
df_diversity_info.dropna(subset=['canonical_name'], inplace=True)
df_class_size_info['canonical_name'] = df_class_size_info['name'].map(name_map)
df_class_size_info.dropna(subset=['canonical_name'], inplace=True)

df_nih_funding # Let's see all rows with duplicate Canonical names
df_nih_funding_consolidated = df_nih_funding.copy()
df_nih_funding_consolidated[df_nih_funding_consolidated.duplicated(subset=['canonical_name'], keep=False)].sort_values(by='canonical_name')
df_nih_funding_consolidated = df_nih_funding_consolidated.groupby('canonical_name', as_index=False).agg({'School of Medicine Award': 'sum'})

hospital_grouped = df_hospital_rankings.groupby('canonical_name').min(numeric_only=True)
hospital_grouped.drop(columns=['#n_ranked_specialties', '#n_top10_specialties', '#n_top1_specialties'], inplace=True, errors='ignore')
hospital_grouped_with_cumulative_columns = hospital_grouped.copy()
hospital_grouped_with_cumulative_columns['#n_ranked_specialties'] = hospital_grouped.count(axis=1)
hospital_grouped_with_cumulative_columns['#n_top10_specialties'] = (hospital_grouped <= 10).sum(axis=1)
hospital_grouped_with_cumulative_columns['#n_top1_specialties'] = (hospital_grouped == 1).sum(axis=1)
hospital_grouped_with_cumulative_columns.loc['Weill Cornell Medicine'] = hospital_grouped_with_cumulative_columns.loc['Columbia University Vagelos College of Physicians and Surgeons']

all_dfs = [
    df_mcat_gpa, 
    df_debt_info, 
    df_nih_funding_consolidated,
    df_faculty_info, 
    hospital_grouped_with_cumulative_columns,
    df_diversity_info,
    df_class_size_info
]

# Drop any rows with 

final_df = reduce(lambda left, right: pd.merge(left, right, on='canonical_name', how='outer'), all_dfs)
final_df.sort_values(by='canonical_name', inplace=True)
columns_we_care_about = ['AAMC_Institution', 'canonical_name', 'Degree Type', 'Average GPA', 'Average MCAT', 
                        'Average Graduate\nIndebtedness','School of Medicine Award', 'Total Faculty',
                        '#n_ranked_specialties', '#n_top10_specialties', '#n_top1_specialties',
                        'URM%',
                        'Applications',	'Class Size', 'Matriculation Rate',
                        'Cancer', 
                        'Cardiology, Heart & Vascular Surgery', 'Diabetes & Endocrinology',
                        'Cardiology, Heart & Vascular Surgery', 'Diabetes & Endocrinology',
                        'Ear, Nose & Throat', 'Gastroenterology & GI Surgery', 'Geriatrics',
                        'Neurology & Neurosurgery', 'Obstetrics & Gynecology', 'Ophthalmology',
                        'Orthopedics', 'Psychiatry', 'Pulmonology & Lung Surgery',
                        'Rehabilitation', 'Rheumatology', 'Urology']
final_df = final_df[columns_we_care_about]
final_df.drop_duplicates(inplace=True)
final_df.rename(columns={'Average Graduate\nIndebtedness': 'Average Graduate Indebtedness', 'School of Medicine Award': 'NIH Research Funding'}, inplace=True)
final_df['NIH Research Funding per Faculty'] = final_df['NIH Research Funding'] / final_df['Total Faculty']
final_df.sort_values(by='NIH Research Funding per Faculty', ascending=False, inplace=True)
final_df.to_csv(os.path.join(ROOT_DIR, 'data/final_medical_school_data.csv'), index=False)
final_df[['AAMC_Institution', 'Average Graduate Indebtedness', 'Matriculation Rate']].head(20)

,AAMC_Institution,Average Graduate Indebtedness,Matriculation Rate
81,Stanford,156377,0.919024
59,NYU-Grossman,67572,1.180680
144,UT Austin-Dell,138150,0.879662
20,Duke,150488,1.332034
49,Mayo,144152,1.841216
105,UC San Francisco,118963,1.706518
153,Washington U St Louis,90880,2.003910
104,UC San Diego,157479,1.346154
15,Columbia-Vagelos,114362,1.797577
148,Vanderbilt,209382,1.236174
